# Lab 1

In [75]:
import numpy as np
from collections import defaultdict

In [76]:
class Component:
    def __init__(self, name : str, node1 : int, node2 : int, current_value : float, initial_value : float):
        self.name = name
        self.node1 = node1
        self.node2 = node2
        self.current_value = current_value
        self.initial_value = initial_value
    
    def __str__(self):
        return f"{self.name} is connected between node{self.node1} and node{self.node2}. Current value is {self.current_value}. Initial value is {self.initial_value}"

In [77]:
def read_file(filePath):
    file = open(filePath, 'r')
    lines = file.readlines()
    
    sim_time = float(lines[0])
    num_iter = float(lines[1])
    nodes = set()
    
    circuit_components = defaultdict(list)
    
    for line in lines[2:-1]:
        name, node1, node2, current_value, initial_value = line.strip('\n').split(' ')
        nodes.add(node1)
        nodes.add(node2)
        
        component = Component(name, int(node1[-1]), int(node2[-1]), float(current_value), float(initial_value))
        circuit_components[name].append(component)
        
    return sim_time, num_iter/sim_time, circuit_components, len(nodes) - 1, len(circuit_components['Vsrc'])

In [78]:
def construct_Gmat(n, resistances):
    G = np.zeros((n, n))
    for res in resistances:
        G[res.node1 - 1][res.node1 - 1] += (1/res.current_value)
        if res.node2 != 0:
            G[res.node2 - 1][res.node2 - 1] += (1/res.current_value)
            G[res.node1 - 1][res.node2 - 1] -= (1/res.current_value)
            G[res.node2 - 1][res.node1 - 1] -= (1/res.current_value)
    return G

In [79]:
def construct_Bmat(n, m, voltage_sources):
    B = np.zeros((n, m))
    for column, voltage_source in enumerate(voltage_sources):
        if voltage_source.node1 != 0: 
            B[voltage_source.node1 - 1][column] = 1
        
        if voltage_source.node2 != 0:
            B[voltage_source.node2 - 1][column] = -1
    return B

In [80]:
def construct_Cmat(b_matrix):
    return b_matrix.transpose()

In [81]:
def construct_Dmat(m):
    return np.zeros((m, m))

In [82]:
def construct_Amat(G, B, C, D):
    G_C = np.concatenate((G, C), axis=0)
    B_D = np.concatenate((B, D), axis=0)
    return np.concatenate((G_C, B_D), axis=1)

In [83]:
def construct_Imat(current_sources, n):
    i = np.zeros((n, 1))
    for cs in current_sources:
        if cs.node1 != 0:
            i[cs.node1 -1] += cs.current_value 
        if cs.node2 != 0:
            i[cs.node2 -1] += cs.current_value
    return i

In [84]:
def construct_Emat(voltage_sources, m):
    e = np.zeros((m, 1))
    for column, voltage_source in enumerate(voltage_sources):
        e[column] = voltage_source.current_value
    return e

In [85]:
def construct_Zmat(i_mat, e_mat):
    return np.concatenate((i_mat,e_mat), axis=0)

In [86]:
# Read all needed data
simulation_time, time_step, circuit_components, n, m =  read_file('./testcases/8.txt')

print(n, m)

2 0


In [87]:
# Construct G matrix
G = construct_Gmat(n, circuit_components['R'])
G

array([[ 2., -1.],
       [-1.,  1.]])

In [88]:
# Construct B matrix
B = construct_Bmat(n, m, circuit_components['Vsrc'])
B

array([], shape=(2, 0), dtype=float64)

In [89]:
# Construct C matrix
C = construct_Cmat(B)
C

array([], shape=(0, 2), dtype=float64)

In [90]:
# Construct D matrix
D = construct_Dmat(m)
D

array([], shape=(0, 0), dtype=float64)

In [91]:
# Construct A matrix
A = construct_Amat(G=G, C=C, B=B, D=D)
A

array([[ 2., -1.],
       [-1.,  1.]])

In [92]:
# Construct i matrix
i = construct_Imat(circuit_components['Isrc'], n)
i

array([[0.],
       [1.]])

In [93]:
# Construct e matrix
e = construct_Emat(circuit_components['Vsrc'], m)
e

array([], shape=(0, 1), dtype=float64)

In [94]:
# Construct Z matrix
Z = construct_Zmat(i, e)
Z

array([[0.],
       [1.]])

In [95]:
# Solve X matrix
inverse_A = np.linalg.inv(A)
X = np.round(np.matmul(inverse_A, Z), 4)
X

array([[1.],
       [2.]])

# Lab 2

In [60]:
def construct_Gmat2(n, h, resistances, capacitors):
    G = construct_Gmat(n=n, resistances=resistances)
    
    # Applying capacitors stamp for backward euler
    for cap in capacitors:
        G[cap.node1 - 1][cap.node1 - 1] += (cap.current_value/h)
        if cap.node2 != 0:
            G[cap.node2 - 1][cap.node2 - 1] += (cap.current_value/h)
            G[cap.node1 - 1][cap.node2 - 1] -= (cap.current_value/h)
            G[cap.node2 - 1][cap.node1 - 1] -= (cap.current_value/h)
    return G

In [61]:
def construct_Imat2(n, current_sources, capaictors):
    i = construct_Imat(n=n, current_sources=current_sources)
    
    # Applying capacitors stamp for backward euler
    for cap in capacitors:
        i[cap.node1 - 1][0] += ( (cap.current_value/h) * cap.initial_value )
        if cap.node2 != 0:
            i[cap.node1 - 1][0] -= ( (cap.current_value/h) * cap.initial_value )
    return i

In [62]:
def construct_Bmat2(n, m, voltage_sources, inductors):
    B_1 = construct_Bmat(n=n, m=m, voltage_sources=voltage_sources)
    
    if len(inductors) > 0:
        B_2 = np.zeros((n, len(inductors)))
        for ind_num, ind in enumerate(inductors):
            if ind.node1 != 0: 
                B_2[ind.node1 - 1][ind_num] = 1
            if ind.node2 != 0:
                B_2[ind.node2 - 1][ind_num] = -1

        B = np.concatenate((B_1, B_2), axis=0)
        return B
    else:
        return B_1

In [63]:
def construct_Dmat2(m, inductors, h):
    D_1 = construct_Dmat(m=m)
    
    if len(inductors) > 0:
        D_2 = np.zeros((len(inductors), len(inductors)))
        for ind_num, ind in enumerate(inductors):
            D_2[ind_num][ind_num] -= (ind.current_value / h)

        D = np.concatenate((D_1, D_2), axis=0)
        return D
    else:
        return D_1

In [64]:
# Construct G matrix
G2 = construct_Gmat2(n=n, resistances=circuit_components['R'], capacitors=circuit_components['C'], h=time_step)
G2

array([[ 2.005, -1.   ],
       [-1.   ,  1.   ]])

In [65]:
# Construct G matrix
B2= construct_Bmat2(n=n, m=m, inductors=circuit_components['I'], voltage_sources=circuit_components['Vsrc'])
B2

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 0 and the array at index 1 has size 1